In [1]:
import os
%pwd

'd:\\Kidney-Disease-classification-cnn\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Kidney-Disease-classification-cnn'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    epochs: int
    batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [5]:
from src.cnn_classifier.constants import *
from src.cnn_classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir, "kidney_dataset")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            epochs=params.EPOCHS,
            batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
        )
        return training_config

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [44]:
%pip install scipy

  Using cached scipy-1.16.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.5 MB ? eta -:--:--
    --------------------------------------- 0.5/38.5 MB 515.6 kB/s eta 0:01:14
    --------------------------------------- 0.5/38.5 MB 515.6 kB/s eta 0:01:14
    --------------------------------------- 0.5/38.5 MB 515.6 kB/s eta 0:01:14
    --------------------------------------- 0.8/38.5 MB 460.6 kB/s eta 0:01:22
    --------------------------------------- 0.8/38.5 MB 460.6 kB/s eta 0:01:22
   - -------------------------------------- 1.0/38.5 MB 501.5 kB/s eta 0:01:15
   - -------------------------------------- 1.0/38.5 MB 501.5 kB/s eta 0:01:15
   - ---------------------

In [8]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        # Load model without optimizer state
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )
        
        # Create a fresh optimizer
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
        # Recompile the model
        self.model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        print("Model loaded and compiled successfully")
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.batch_size,
            interpolation="bilinear",
            class_mode="categorical"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        # OPTIMIZED FOR CPU - DRAMATICALLY REDUCE STEPS
        self.steps_per_epoch = 80    # Instead of 622!
        self.validation_steps = 20   # Instead of 77!
        
        print(" OPTIMIZED CPU TRAINING STARTED")
        print(f"Steps per epoch: {self.steps_per_epoch} (was 622)")
        print(f"Validation steps: {self.validation_steps} (was 77)")
        print(f"Expected time per epoch: ~20-30 minutes")
        print(f"Total expected time: 5-7 hours for {self.config.epochs} epochs")
        
        # Add early stopping to potentially stop even earlier
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_accuracy',
                patience=3,
                restore_best_weights=True,
                verbose=1
            ),
            tf.keras.callbacks.ModelCheckpoint(
                filepath=str(self.config.trained_model_path).replace('.keras', '_best.keras'),
                save_best_only=True,
                monitor='val_accuracy',
                mode='max',
                verbose=1
            )
        ]

        # Train the model with optimized steps
        history = self.model.fit(
            self.train_generator,
            epochs=self.config.epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callbacks,
            verbose=1
        )

        # Save the final model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        
        return history

In [9]:
import tensorflow as tf
import os
import time


# Configure environment for CPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')

# Clear session
tf.keras.backend.clear_session()

start_time = time.time()

try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    
   
    
    print(f" OPTIMIZED TRAINING CONFIGURATION:")
    print(f"Epochs: {training_config.epochs}")
    print(f"Batch Size: {training_config.batch_size}")
    
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    
    # Start optimized training
    training.train()
    
    total_time = time.time() - start_time
    hours = total_time // 3600
    minutes = (total_time % 3600) // 60
    
    print(f"Training completed in {int(hours)}h {int(minutes)}m")
    
except Exception as e:
    print(f" Training failed: {e}")
    import traceback
    traceback.print_exc()

[2025-10-31 22:10:57,650 - cnn_classifier - INFO - yaml file:config\config.yaml loaded successfully]
[2025-10-31 22:10:57,662 - cnn_classifier - INFO - yaml file:params.yaml loaded successfully]
[2025-10-31 22:10:57,666 - cnn_classifier - INFO - created directory at:artifacts]
[2025-10-31 22:10:57,673 - cnn_classifier - INFO - created directory at:artifacts\training]
 OPTIMIZED TRAINING CONFIGURATION:
Epochs: 15
Batch Size: 32
Model loaded and compiled successfully
Found 1471 images belonging to 2 classes.
Found 5889 images belonging to 2 classes.
 OPTIMIZED CPU TRAINING STARTED
Steps per epoch: 80 (was 622)
Validation steps: 20 (was 77)
Expected time per epoch: ~20-30 minutes
Total expected time: 5-7 hours for 15 epochs


d:\Kidney-Disease-classification-cnn\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 557s/step - accuracy: 0.6734 - loss: 0.8392  

d:\Kidney-Disease-classification-cnn\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 